In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import pandas as pd
from PIL import Image as Img
import random
from skimage import color
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader


In [ ]:
main_dir = '/kaggle/input/uw-madison-gi-tract-image-segmentation/'


Note: I got this idea from https://www.kaggle.com/code/fabiendaniel/image-with-masks-quick-overview

In [ ]:


df_train = pd.read_csv(main_dir + 'train.csv')

# name = df_train[200]

# target = name.split('/')[-3] + '_slice_' + name.split('/')[-1].split('_')[1]
# # print(name.split('/')[-3] + '_slice_' + name.split('/')[-1].split('_')[1])
# # print(df_train.index[df_train['id']==target].tolist())
# mask = decode_rle_mask(str(df_train.index[df_train['id']==target].tolist()[1]), (266,266))

df_train.sample(5)


In [ ]:
from glob import glob
list_images = glob('../input/uw-madison-gi-tract-image-segmentation/train/*/*/scans/*.png')
# print(list_images[0])

image_properties = pd.DataFrame([(c, c.split('/')[-3], c.split('/')[-1]) for c in list_images], columns = ['whole_path', 'case_day', 'file'])

image_properties.sample(5)

In [ ]:
image_properties['slice'] = image_properties['file'].apply(lambda x: f"slice_{x.split('_')[1]}")
image_properties.sample(5)

In [ ]:
image_properties['height'] = image_properties['file'].apply(lambda x: int(x.split('_')[2]))
image_properties['width']  = image_properties['file'].apply(lambda x: int(x.split('_')[3]))
image_properties.sample(5)

In [ ]:
image_properties['id']= image_properties['case_day'] + '_' + image_properties['slice']
image_properties.sample(5)

In [ ]:
df_train = pd.merge(df_train, image_properties, on='id', how='left')
df_train.sample(3)

# visualize some of the data

In [ ]:
nx, ny = 6, 2
fig, axes = plt.subplots(ny, nx, figsize=(24, 9))
images = random.choices(list_images, k=nx*ny)
# print(images[0])
for num, img in enumerate(images):
    i = num % nx
    j = num // nx
    image = Img.open(img)
    image = np.array(image)
#     iar_shp = np.array(image).shape
    axes[j, i].axis('off')
#     axes[j, i].set_title(str(img.parent).strip('/scans').split('/')[-1], color='red')
    axes[j, i].imshow(image / image.max())
    
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()

In [ ]:
def rle_decode(rle, height, width , fill=255):
    s = rle.split()
    start, length = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    start -= 1
    mask = np.zeros(height*width, dtype=np.uint8)
    for i, l in zip(start, length):
        mask[i:i+l] = fill
    mask = mask.reshape(width,height).T
    mask = np.ascontiguousarray(mask)
    return mask

In [ ]:
X = df_train[df_train['segmentation'].notnull()]

nx, ny = 3, 4
fig, axes = plt.subplots(ny, nx, figsize=(20, 15))


for i in range(ny):
    print(random.randint(0, X.shape[0]))
    sample = X.iloc[random.randint(0, X.shape[0])]
#     print(sample.head())
#     height, width = 255, 255
    mask = rle_decode(sample['segmentation'], sample['height'], sample['width'], 255)
#     mask = decode_rle_mask(sample['segmentation'], [sample['height'], sample['width']])
    mask = (mask / 255).astype(int).T

    if i == 0 : axes[i, 0].set_title("mask", color='red')
    axes[i, 0].set_ylabel(sample['id'])
    axes[i, 0].imshow(mask)
    image = Img.open(sample['whole_path'])
    if i == 0 : axes[i, 1].set_title("image", color='red')
    axes[i, 1].imshow(np.array(image))

    result_image = color.label2rgb(mask, np.array(image) / np.array(image).max())
    if i == 0 : axes[i, 2].set_title("mask_image overlay", color='red')
    axes[i, 2].imshow(result_image)

# PyTorch dataLoader

In [ ]:
#!/usr/bin/env python3
# encoding: utf-8


class UW_Madison(Dataset):

    def __init__(self, patients_dataframe, crop_size=None, modes=None, train=True):
        self.patients_dataframe = patients_dataframe

        self.modes = modes
        self.train = train
        self.crop_size = crop_size

    def __len__(self):
        return len(self.patients_dir)

    def __getitem__(self, index):
        sample = X.iloc[index]
        mask = rle_decode(sample['segmentation'], sample['height'], sample['width'], 255)
        mask = (mask / 255).astype(int).T
        image = Img.open(sample['whole_path'])


#         patient_dir = self.patients_dir[index]
#         image = cv2.imread(patient_dir)

        return image, mask


In [ ]:
patients_dataframe = df_train[df_train['segmentation'].notnull()]

data_loader = UW_Madison(patients_dataframe)

fig , axes = plt.subplots(1, 3, figsize=(20, 15))
import numpy as np
image , mask = data_loader[0]
# print(np.unique(mask))
axes[0].imshow(np.array(image))
axes[0].set_title('image')

axes[1].imshow(mask)
axes[1].set_title('mask')

merged_image = color.label2rgb(mask, np.array(image) / np.array(image).max())
axes[2].set_title('merged_image')
axes[2].imshow(merged_image)



# Next version:

    1. adding data_augmentation
    2. training loop